In [22]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [23]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [24]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [25]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [26]:
# Set features. This will also be used as your x values.
#selected_features = df[['names', 'of', 'selected', 'features', 'here']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [29]:
X =df.drop('koi_disposition', axis =1)
y = df['koi_disposition']
print(X.shape, y.shape)

(6991, 40) (6991,)


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [31]:
# Scale your data
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler

# define min max scaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [32]:
 X_train_scaled

array([[1.        , 0.        , 0.        , ..., 0.42898713, 0.49393616,
        0.50318208],
       [0.        , 0.        , 0.        , ..., 0.62560683, 0.28310341,
        0.67741136],
       [0.        , 1.        , 0.        , ..., 0.13460577, 0.44064722,
        0.59889247],
       ...,
       [0.        , 1.        , 0.        , ..., 0.73068093, 0.51842507,
        0.4956608 ],
       [0.        , 0.        , 0.        , ..., 0.71647611, 0.36021512,
        0.22993636],
       [0.        , 0.        , 0.        , ..., 0.58526391, 0.29853405,
        0.65798826]])

# Train the Model



In [33]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier

model_svc = SVC(kernel='linear')
model_svc.fit(X_train_scaled, y_train)

SVC(kernel='linear')

In [34]:
# Model Accuracy
print(f"Training Data Score: {model_svc.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model_svc.score(X_test_scaled, y_test)}")

Training Data Score: 0.8472248712569139
Testing Data Score: 0.852974828375286


In [35]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model_svc.predict(X_test_scaled)
print(classification_report(y_test,predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.76      0.61      0.68       422
     CONFIRMED       0.69      0.79      0.74       450
FALSE POSITIVE       0.99      1.00      0.99       876

      accuracy                           0.85      1748
     macro avg       0.81      0.80      0.80      1748
  weighted avg       0.85      0.85      0.85      1748



# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [37]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid_search = GridSearchCV(model_svc, param_grid, verbose=3)

In [38]:
# Train the model with GridSearch
# Fit the grid search to the data
grid_search.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END .................C=1, gamma=0.0001;, score=0.848 total time=   0.3s
[CV 2/5] END .................C=1, gamma=0.0001;, score=0.840 total time=   0.2s
[CV 3/5] END .................C=1, gamma=0.0001;, score=0.827 total time=   0.2s
[CV 4/5] END .................C=1, gamma=0.0001;, score=0.849 total time=   0.3s
[CV 5/5] END .................C=1, gamma=0.0001;, score=0.844 total time=   0.2s
[CV 1/5] END .................C=1, gamma=0.0005;, score=0.848 total time=   0.2s
[CV 2/5] END .................C=1, gamma=0.0005;, score=0.840 total time=   0.2s
[CV 3/5] END .................C=1, gamma=0.0005;, score=0.827 total time=   0.2s
[CV 4/5] END .................C=1, gamma=0.0005;, score=0.849 total time=   0.2s
[CV 5/5] END .................C=1, gamma=0.0005;, score=0.844 total time=   0.2s
[CV 1/5] END ..................C=1, gamma=0.001;, score=0.848 total time=   0.3s
[CV 2/5] END ..................C=1, gamma=0.001;

GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             verbose=3)

In [39]:
# List the best parameters for this dataset
print(grid_search.best_params_)
print(grid_search.best_score_)

{'C': 50, 'gamma': 0.0001}
0.8840364141785344


In [40]:
grid_search.score(X_train_scaled, y_train)

0.8863246233072668

In [41]:
grid_search.score(X_test_scaled, y_test)

0.8913043478260869

In [42]:
# Make predictions with the hypertuned model
predictions = grid_search.predict(X_test_scaled)

# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.84      0.71      0.77       422
     CONFIRMED       0.76      0.86      0.80       450
FALSE POSITIVE       0.99      1.00      0.99       876

      accuracy                           0.89      1748
     macro avg       0.86      0.85      0.86      1748
  weighted avg       0.89      0.89      0.89      1748



# Save the Model

In [43]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'Eric.sav'
joblib.dump(model_svc, filename)

['Eric.sav']